In [1]:
# synaptic bank

from carabao.neuron import Synapses,toy
k,g,K,P,c = toy('cell')
syn = Synapses(K,P)
c = [1,1,1,1,1,1,1,1,1,1]; print("c:",repr(c)) 
P=syn.P;    print("syn.P   ",repr(P))
W=syn.W();  print("syn.W() ",repr(W))
V=syn.V(c); print("syn.V(c)",repr(V)) 
E=syn.E(c); print("syn.E(c)",repr(E))
S=syn.S(c); print("syn.S(c)",repr(S))
L=syn.L(c); print("syn.L(c)",repr(L))
s=syn.s(c); print("syn.s(c)",repr(s))
Q1=syn.sat(1.5*P);  print("Q1",repr(Q1))
Q2=syn.sat(P-0.7*Q1);  print("Q2",repr(Q2))

c: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
syn.P    array([[0.12, 0.32, 0.54, 0.77, 0.  ],
       [0.  , 0.61, 0.45, 0.  , 0.8 ]])
syn.W()  array([[0, 0, 1, 1, 0],
       [0, 1, 0, 0, 1]])
syn.V(c) array([[1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1]])
syn.E(c) array([[0, 0, 1, 1, 0],
       [0, 1, 0, 0, 1]])
syn.S(c) array([[1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1]])
syn.L(c) array([[0.02, 0.02, 0.02, 0.02, 0.02],
       [0.02, 0.02, 0.02, 0.02, 0.02]])
syn.s(c) array([1, 1])
Q1 array([[0.18 , 0.48 , 0.81 , 1.   , 0.   ],
       [0.   , 0.915, 0.675, 0.   , 1.   ]])
Q2 array([[0.  , 0.  , 0.  , 0.07, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.1 ]])


In [2]:
# synaptic field

from carabao.cell import toy
k,g,K,P,c = toy('cell')
syn = Synapses(g)
c = [0,1,0,1,0,0,0,0,0,0]; print("c:",repr(c)) 
v=syn.v(c); print("syn.v(c)",repr(v)) 


c: [0, 1, 0, 1, 0, 0, 0, 0, 0, 0]
syn.v(c) array([0, 1, 0, 1])


In [8]:
help(Synapses)

Help on class Synapses in module carabao.neuron:

class Synapses(builtins.object)
 |  Synapses(K, P=None, eta=0.5, theta=2, delta=(0.02, 0.02))
 |  
 |  Synapses: class
 |      syn = Synapses(K,P,eta,theta,(plus,minus))   # full arg list
 |      syn = Synapses(K,P)       # eta=0.5, plus=minus=0.02
 |      syn = Synapses(g)         # P=one
 |  
 |      P = syn.P                 # permanences
 |  
 |      W = syn.W()               # synaptic weight matrix
 |      V = syn.V(c)              # presynaptic signals
 |      E = syn.E(c)              # empowering matrix
 |      L = syn.L(c)              # learning mask
 |      D = syn.D(c)              # learning deltas
 |      s = syn.s(c)              # spike vector
 |      o = syn.one               # [1,1,...,1] matrix (1 x ns)
 |  
 |      syn.P = syn.sat(P)        # truncate P matrix to range [0,1]
 |  
 |  Methods defined here:
 |  
 |  E(self, c)
 |  
 |  L(self, c)
 |  
 |  S(self, c)
 |  
 |  V(self, c)
 |  
 |  W(self)
 |  
 |  __init